In [ ]:
from birdset.datamodule import DatasetConfig
from birdset.datamodule.birdset_datamodule import BirdSetDataModule
from birdset.datamodule.components.transforms import BirdSetTransformsWrapper
from transformers import Wav2Vec2Model, HubertModel, Trainer
import torch
from torchmetrics import Accuracy, Precision
import time
import fairseq
from tqdm import tqdm
import lightning as lt
from huggingface_hub import PyTorchModelHubMixin
from lightning.pytorch.loggers import TensorBoardLogger
from torchmetrics.classification.average_precision import AveragePrecision
from torchmetrics.classification.auroc import AUROC
from sklearn.metrics import average_precision_score
import time, os

In [ ]:
model_name_base = "facebook/hubert-base-ls960"
model_name_large = "facebook/hubert-large-ll60k"
model_name = model_name_large
basemodel = Wav2Vec2Model.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation=None)

In [ ]:
from Model import Modelwrapper

model = Modelwrapper.load_model("D:\\Documents\\Uni\DLL\\Code\models\\Modelwrapper06-10-2024-21-00-09_large50.pt", basemodel)

In [ ]:
transforms = BirdSetTransformsWrapper(
        task="multilabel",
        sampling_rate = 16000,
        model_type = 'waveform',
    )

dataset_name = "HSN"
dm = BirdSetDataModule(
    dataset= DatasetConfig(
        data_dir='B:\DLL\Datasets',
        dataset_name= "",
        hf_path='DBD-research-group/BirdSet',
        hf_name=dataset_name,
        n_workers=8,
        val_split=0.2,
        task="multilabel",
        classlimit=500,
        eventlimit=1,
        sampling_rate=16000,
    ),
    transforms=transforms
)

dm.prepare_data()
dm.setup(stage="test")
test_dataloader = dm.test_dataloader()
print(type(test_dataloader.dataset))
print(test_dataloader.dataset)
print(type(test_dataloader))

In [ ]:
batch = next(iter(test_dataloader))
print(batch["input_values"])

In [ ]:
print(model.get_proba_sigmoid(batch))
print(batch["labels"])